[점프투파이썬_예외처리](https://wikidocs.net/30)

### try/finally
try블록 수행 중 오류 발생 여부와 상관없이 finally블록은 항상 실행된다.

<br>
파일을 열 때 발생하는 오류는 호출한 쪽에 전달되어야 하므로<br>
try블록 앞에서 open을 호출한다.

In [13]:
def try_finally_example(filename):
    print('* 파일 열기')
    handle = open(filename, encoding='utf-8') # OSError 발생할 수 있음
    try:
        print('* 데이터 읽기')
        return handle.read()      # UnicodeDecodeError 발생할 수 있음
    finally:
        print('* close() 호출')
        handle.close()            # try 블록이 실행된 다음에는 항상 이 블록이 실행됨


filename = 'random_data.txt'

# 이진문자열을 쓰기: 'wb'
with open(filename, 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')  # 잘못된 utf-8 이진 문자열

# 데이터를 읽을 때 오류 발생
# try-finally
#data = try_finally_example(filename)

# 파일을 열 때 오류 발생
# finally를 거치지 않는다.
try_finally_example('does_not_exist.txt')

* 파일 열기


FileNotFoundError: [Errno 2] No such file or directory: 'does_not_exist.txt'

### try/except/else
except 발생오류 as 오류메시지 변수
-  except문에 미리 정해 놓은 오류 이름과 일치할 때만 except 블록을 수행한다

<br>
else블록은 오류가 없을 경우에만 수행된다.

- else블록을 사용하면 try블록 안에 들어갈 코드를 최소화할 수 있다.
- try블록에 들어가는 코드가 줄어들면 **발생할 여지가 있는 예외를 서로 구분할 수 있으므로 가독성이 좋아진다.**

In [15]:
import json

def load_json_key(data, key):
    try:
        print('* JSON 데이터 읽기')
        result_dict = json.loads(data)     # ValueError가 발생할 수 있음
    except ValueError as e:
        print('* ValueError 처리')
        raise KeyError(key) from e
    else:
        print('* 키 검색')
        return result_dict[key]            # KeyError가 발생할 수 있음

# 성공적으로 실행
# try-else
assert load_json_key('{"foo": "bar"}', 'foo') == 'bar'

# 입력이 올바른 JSON이 아니므로 json.loads가 디코딩하는 중에 ValueError 발생
# try-except
#load_json_key('{"foo": bad payload', 'foo')         # 잘못된 데이터형식

# 키 검색에서 예외 발생
# try-else
load_json_key('{"foo": "bar"}', '존재하지 않음')   # 존재하지 않는 키 입력

* JSON 데이터 읽기
* 키 검색
* JSON 데이터 읽기
* 키 검색


KeyError: '존재하지 않음'

### try/except/else/finally
- 이 구조는 모든 블럭이 직관적으로 작동하기 때문에 유용하다.

json데이터에 저장된 numerator와 denominator를 나눈 결과를 다시 json에 저장하는 코드

In [5]:
UNDEFINED = object()

def divide_json(path):
    print('* 파일 열기')
    handle = open(path, 'r+')             # OSError가 발생할 수 있음
    try:
        print('* 데이터 읽기')
        data = handle.read()              # UnicodeDecodeError가 발생할 수 있음
        print('* JSON 데이터 읽기')
        op = json.loads(data)             # ValueError가 발생할 수 있음
        print('* 계산 수행')
        value = (
            op['numerator'] /
            op['denominator'])            # ZeroDivisionError가 발생할 수 있음
    except ZeroDivisionError as e:
        print('* ZeroDivisionError 처리')
        return UNDEFINED
    else:
        print('* 계산 결과 쓰기')
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)                    # OSError가 발생할 수 있음
        handle.write(result)              # OSError가 발생할 수 있음
        return value
    finally:
        print('* close() 호출')
        handle.close()                    # 어떤 경우든 실행됨

temp_path = 'random_data.json'

with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 10}')

# 정상적인 경우
# try-else-finally블록 수행 
assert divide_json(temp_path) == 0.1 # 계산 결과

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* 계산 수행
* 계산 결과 쓰기
* close() 호출


In [6]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 0}')

# 계산이 잘못된 경우
# try-except-finally
assert divide_json(temp_path) is UNDEFINED

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* 계산 수행
* ZeroDivisionError 처리
* close() 호출


In [7]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1 bad data')

# JSON데이터가 잘못된 경우
# try-finally
divide_json(temp_path)

* 파일 열기
* 데이터 읽기
* JSON 데이터 읽기
* close() 호출


JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)